In [1]:
import requests
from bs4 import BeautifulSoup


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup, Tag, NavigableString

In [3]:
import os 
import re
import pandas as pd
import numpy as np

In [4]:
#OCCHIO CHE CON RELATION HA LA PUNTEGGIATURA PER FARE GLI SPLIT, MA IL POSTAGGING E' SBAGLIATO!!!!!!!!!!
dir_lis = os.listdir('output_pedalion_con_relation')

# Lista per salvare tutti i percorsi dei file CSV
csv_file_paths = []

# Directory principale che contiene le sottodirectory
main_directory = 'output_pedalion_con_relation'

#per ogni cartella in output gorman devo ottenere la directory completa della singola cartella dell'autore. per farlo itero col ciclo for su tutte le cartelle presenti in gorman e concateno,
#per ottenere la directory della cartella della singola opera, la main directory col nome della cratella.
#per farlo uso la funzione di os che si chiama os.path join che fa una concatenazione di stringhe.
for folder in dir_lis:
  folder_path = os.path.join(main_directory, folder)
  #check is folder_path is folder dir
  if os.path.isdir(folder_path):  #ora voglio per ogni cartella una lista vuota poi da andare a riempire con i csv di ogni singola cartella.
    singola_cartella = []
    for file in os.listdir(folder_path): #per ogni file che sta nella mia cartella
      if file.endswith('.csv'): #controllo per vedere se sono davvero tutti csv i file.
        file_path = os.path.join(folder_path, file) #se il file è un csv la sua directory completa me la metti nella lista.
        singola_cartella.append(file_path)

  csv_file_paths.append(singola_cartella)
csv_file_paths = sorted(csv_file_paths)
# Lista per conservare i DataFrame filtrati
#per ogni lista creo una lista di di dataframe. questi sono i csv filtrati con le info che mi servono.
#ora lo trasformo in una funzione che prenda in input una lista di csv.
def filter_dataframe(csv_file_paths):
  dataframes_tot = [] #è una lista vuota in cui mettere i dataframe filtrati.
# Leggi, filtra e aggiungi ogni DataFrame alla lista
  for file_path in csv_file_paths:
    # Leggi il CSV in un DataFrame
      df = pd.read_csv(file_path)

    # Filtra il DataFrame mantenendo solo le righe dove '@lemma' è una stringa
      df_filtered = df[df['@lemma'].apply(lambda x: isinstance(x, str))]

    # Aggiungi il DataFrame filtrato alla lista
      dataframes_tot.append(df_filtered)

# Visualizza le prime righe di ciascun DataFrame per controllare la struttura
      data_info = {f"File {i+1}": df.head() for i, df in enumerate(dataframes_tot)}

# Mostra la struttura dei dati
      #for file_name, df_head in data_info.items():
       # print(f"\n{file_name}:")
        #print(df_head)
  return dataframes_tot
# Function to extract the numeric part from the filename
def extract_number(file_path):
    # Use regex to find the numeric part of the filename
    return int(re.search(r'(\d+)\.csv$', file_path).group(1))
all_sorted_csv_filepaths = []
for folder in csv_file_paths:
  #print(folder)
  sorted_files = sorted(folder, key=extract_number)
  all_sorted_csv_filepaths.append(sorted_files)


In [5]:
dataframes = [filter_dataframe(lis) for lis in all_sorted_csv_filepaths]


In [6]:
list_dir = os.listdir('output_pedalion_con_relation')
all_opera = {}
for i,opera in enumerate(list_dir):
    full_text = ''
    for sent in dataframes[i]:
        try:
            full_text += ' '.join(sent['@form'])
            all_opera[opera] = full_text
        except:
            print('errore')
            print(sent)

In [7]:
all_opera.keys()

dict_keys(['aeneas.xml', 'aesop1.xml', 'charb1.xml', 'chion.xml', 'crit.xml', 'epictetus', 'epictetus.xml', 'epicurus1.xml', 'genesis1.xml', 'genesis2.xml', 'genesis3.xml', 'heron.xml', 'iso.xml', 'julian.xml', 'longus.xml', 'lucian_lis.xml', 'lucian_prometheus.xml', 'lucian_symposion.xml', 'lysias_or24.xml', 'paean.xml', 'papyri.xml', 'phlegon.xml', 'procopius.xml', 'pseudo-lucian_themule.xml', 'pseudoplato_cleitophon.xml', 'sextus.xml', 'theophr.xml', 'xenmem.xml'])

In [9]:
all_opera = {}
for i,opera in enumerate(list_dir):
    all_data = pd.concat(dataframes[i],ignore_index=True)
   
    #for sent in dataframes[i]:
    full_text = ' '.join(all_data['@form'])
    all_opera[opera] = full_text

In [10]:
opera_names = list(all_opera.keys())

In [11]:
#STEP ci indica il numero di frasi per chunk 
STEP = 18
all_opera_split = {}
for key in opera_names:
    joint_text = []
    text = all_opera[key]
    sents = text.split('.')
    for i in range(0, len(sents), STEP):
        joint_text.append('.'.join(sents[i:i+STEP])+'.')
    all_opera_split[key] = joint_text

In [12]:
len(opera_names)

28

In [13]:
#stampare i vari chunks in celle separate  
from IPython.display import display
num = 0
for i in range(len(all_opera_split[opera_names[num]])):
    display(opera_names[num])
    display(all_opera_split[opera_names[num]][i])

'aeneas.xml'

'ὅσοις τῶν ἀνθρώπων ἐκ τῆς αὑτῶν ὁρμωμένοις χώρας ὑπερόριοί τε ἀγῶνες καὶ κίνδυνοι συμβαίνουσιν , ἄν τι σφάλμα γένηται κατὰ γῆν ἢ κατὰ θάλασσαν , ὑπολείπεται τοῖς περιγιγνομένοις αὐτῶν οἰκεία τε χώρα καὶ πόλις καὶ πατρίς , ὥστε οὐκ ἂν ἄρδην πάντες ἀναιρεθείησαν · τοῖς δὲ ὑπὲρ τῶν μεγίστων μέλλουσι κινδυνεύειν , ἱερῶν καὶ πατρίδος καὶ γονέων καὶ τέκνων καὶ τῶν ἄλλων , οὐκ ἴσος οὐδὲ ὅμοιος ἀγών ἐστιν , ἀλλὰ σωθεῖσι μὲν καὶ καλῶς ἀμυναμένοις τοὺς πολεμίους φοβεροὺς τοῖς ἐναντίοις καὶ δυσεπιθέτους εἰς τὸν λοιπὸν χρόνον εἶναι , κακῶς δὲ προσενεχθεῖσι πρὸς τοὺς κινδύνους οὐδεμία ἐλπὶς σωτηρίας ὑπάρξει . τοὺς οὖν ὑπὲρ τοσούτων καὶ τοιούτων μέλλοντας ἀγωνίζεσθαι οὐδεμιᾶς παρασκευῆς καὶ προθυμίας ἐλλιπεῖς εἶναι δεῖ , ἀλλὰ πολλῶν καὶ παντοίων ἔργων πρόνοιαν ἑκτέον , ὅπως διά γε αὑτοὺς μηδὲν φανῶσι σφαλέντες · ἂν δὲ ἄρα τι σύμπτωμα γένηται , ἀλλ’ οἵ γε λοιποὶ τὰ ὑπάρχοντα εἰς τ- αὐτό ποτε καταστήσαιεν ἄν , καθάπερ τινὲς τῶν Ἑλλήνων εἰς τὸ ἔσχατον ἀφικόμενοι πάλιν ἀνέλαβον ἑαυτούς . τὴν οὖν τῶν σω

'aeneas.xml'

' ὡς δὲ αὕτως καὶ κατὰ τῶν ἄλλων πάντων θελημάτων χρὴ τὰ ἐνόντα ὑπεναντία τοῖς προγεγραμμένοις ὑπονοεῖν , ἵνα μὴ ἀπερισκέπτως τι ἕτερον αἱρῇ . ἐκ προσφάτου δὲ ἐγγιγνομένου φόβου ἀσυντάκτῳ πόλει , τάχιστα ἄν τις εἰς σύνταξιν καὶ φυλακὴν τῆς πόλεως τοὺς πολίτας καταστήσαι , εἰ ἑκάστῃ φυλῇ μέρος τι τοῦ τείχους κλήρῳ ἀποδείξειεν , ἐφ’ ᾧ ἐλθοῦσαι εὐθὺς αἱ φυλαὶ φυλάξουσιν . κατὰ πολυπλήθειαν δὲ φυλῆς ἑκάστης τὸ μέγεθος τοῦ τείχους φυλαττόντων . ἔπειτα οὕτως ἀφ’ ἑκάστης φυλῆς τοὺς δυναμένους τοῖς σώμασι πονέσαι ἀπολέγειν εἴς τε τὴν ἀγορὰν καὶ τὰς περιοδίας , καὶ εἴ τι ἄλλο δεῖ χρῆσθαι τοῖς τοιούτοις ἀνθρώποις . ὁμοτρόπως δὲ καὶ φρουρίου ὑπὸ συμμάχων φρουρουμένου μέρος τι τοῦ τείχους τῶν συμμάχων ἑκάστοις ἀποδιδόσθω φυλάττειν . ἐὰν δὲ πολῖται ἐν ὑποψίᾳ πρὸς ἀλλήλους ὦσιν , κατὰ ἀνάβασιν ἑκάστην τοῦ τείχους δεῖ ἐπιστῆσαι ἄνδρας πιστούς , οἳ κωλυταὶ ἔσονται , ἄν τις ἐπιχειρῇ ἄλλος ἀναβαίνειν . ἐν εἰρήνῃ δὲ καὶ ὧδε χρὴ συντετάχθαι τοὺς πολίτας . πρῶτον μὲν ῥύμης ἑκάστης ἀποδεῖξαι ῥυμάρχην ἄνδρα 

'aeneas.xml'

...

' ὅταν γὰρ ἐπὶ τὰς πύλας τις αὐτῶν ἐλθὼν κλεῖσαι μὴ αὐτουργὸς γίγνηται , ἀλλὰ παραδοὺς τὴν βάλανον τῷ πυλωρῷ κλεῖσαι κελεύῃ , τάδε κακουργεῖται ὑπὸ πυλωρῶν βουλομένων νυκτὸς δέξασθαι τοὺς πολεμίους . ὁ μέν τις ἡμέρας εἰς τὴν βαλανοδόκην τῶν πυλῶν ἄμμον προενέβαλεν , ὅπως ἡ βάλανος ἔξω μένῃ καὶ μὴ ἐμβάλληται εἰς τὸ τρύπημα . φασὶ δὲ καὶ ἐμβεβλημένας βαλάνους ὧδε ἐξαιρεθῆναι . ἐμβαλλομένης κατ’ ὀλίγον ἄμμου εἰς τὴν βαλανοδόκην σείεσθαι ἀψοφητί , ἵνα μηδεὶς αἴσθηται . μετέωρος οὖν ἡ βάλανος ἐγίγνετο προσπιπτούσης τῆς ψάμμου , ὥστε ῥᾳδίως αὐτὴν ἐξαιρεθῆναι . ἤδη δὲ πυλωρὸς δεξάμενος τὴν βάλανον παρὰ στρατηγοῦ ἐμβαλεῖν , ἐντεμὼν λάθρᾳ σμίλῃ ἢ ῥίνῃ τὴν βάλανον , βρόχον λίνου περιβαλὼν ἐνέβαλεν , εἶτα μετ’ ὀλίγον τῷ λίνῳ ἀνέσπασεν . ἄλλος δὲ προετοιμάσας ἐν γυργάθῳ λεπτῷ ἐνέβαλεν προσημμένου λίνου , καὶ ὕστερον ἀνέσπασεν . ἐξῃρέθη δὲ καὶ ἀνακρουσθεῖσα ἡ βάλανος . ἔτι δὲ καὶ θερμαστίῳ λεπτῷ ἐξῃρέθη · χρὴ δὲ τοῦ θερμαστίου τὸ μὲν ἓν μέρος εἶναι οἷον σωλῆνα , τὸ δὲ ἕτερον πλατύ , ὥστε τῷ μὲν σωλ

'aeneas.xml'

' πηλὸν κεραμικὸν περιελίξαντες ὀθονίῳ λεπτῷ καθῆκαν , ἀρμένῳ πιέζοντες περὶ τὴν βάλανον τὸν πηλόν · ἔπειτα ἀνέσπασαν τὸν πηλὸν καὶ ἔλαβον τὸν τύπον τῆς βαλάνου , πρὸς ὃν τὴν βαλανάγραν ἐποιήσαντο . συμβάλλεται γενέσθαι Τημένῳ Ῥοδίῳ ἐν Ἰωνίᾳ Τέως πόλις εὐμεγέθης πρόδοτος ὑπὸ τοῦ πυλωροῦ . ἄλλα τε οὖν προσυνέθεντο καὶ νύκτα ἀσέληνον καὶ σκοτεινήν , ἐν ᾗ ἔδει τὸν μὲν ἀνοῖξαι , τὸν δὲ μετὰ ξένων εἰσελθεῖν . ἐπεὶ δ’ ἔδει εἰς τὴν ἐπιοῦσαν νύκτα πραχθῆναι , παρῆν τῷ πυλωρῷ ἀνὴρ ὅστις ἐπεὶ ὀψὲ ἦν καὶ φυλακαί τε ἐπὶ τοῦ τείχεος καθίσταντο καὶ αἱ πύλαι ἔμελλον κλείεσθαι , σκότους οὖν ἤδη ὄντος , ἀπηλλάσσετο , ἐκδήσας ἀρχὴν ἀγαθίδος λίνου κλωστοῦ , ὅπερ οὐκ ἔμελλεν ῥᾳδίως διαρραγῆναι . τὴν δὲ ἀγαθίδα ἐπορεύθη ἀπελίσσων ἀπὸ τῆς πόλεως πέντε στάδια , ὅπῃ ἔμελλον οἱ εἰσπορευσόμενοι ἥξειν . ἐπεὶ δὲ παρῆν ὁ στρατηγὸς κλεῖσαι τὰς πύλας , καὶ ἔδωκε κατὰ τὸ ἔθος τῷ πυλωρῷ ἐμβαλεῖν τὴν βάλανον , δεξάμενος ἐνέτεμεν λαθραίως ἀψοφητὶ ῥίνῃ ἢ σμίλῃ τὴν βάλανον , ὥστε ἐνέχεσθαι λίνον . εἶτα βρόχον περιθεὶς καθ

'aeneas.xml'

' G? πράσσοντα δέ τι τοιοῦτον τὸν μοχλὸν ἀφανίζειν · ἤδη γάρ τινες ἐπιφανέντες ὑπεναντίοι ἐβιάσαντο πάλιν κλεῖσαι , τοῦ μοχλοῦ παρόντος . διὸ δεῖ πάντα τὰ τοιαῦτα προνοεῖν . διαπρίοντα δὲ μοχλὸν ἔλαιον ἐπιχεῖν · θᾶσσον γὰρ καὶ ἀψοφητὶ μᾶλλον πρισθήσεται . ἐὰν δὲ καὶ σπόγγος ἐπί τε τὸν πρίονα καὶ τὸν μοχλὸν ἐπιδεθῇ , πολλῷ κωφότερος ὁ ψόφος ἔσται . πολλὰ δ’ ἄν τις καὶ ἄλλα ὁμότροπα τούτοις γράψαι . ἀλλὰ ταῦτα μὲν παρετέον . εἰς δὲ τὸ τούτων μηδὲν κακουργεῖσθαι χρὴ , πρῶτον μὲν στρατηγὸν μὴ δεδειπνηκότα δι’ αὑτοῦ τὴν κλεῖσιν καὶ τὴν ἐπιμέλειαν ποιεῖσθαι , μηδὲ ἄλλῳ πιστεύειν ῥᾳθύμως διακείμενον · ἐν δὲ τοῖς ἐπικινδύνοις καὶ πάμπαν σύννουν δεῖ περὶ ταύτην εἶναι . ἔπειτα τὸν μοχλὸν σεσιδηρῶσθαι διὰ μήκους τριχῇ ἢ τετραχῇ · ἄπριστος γὰρ ἔσται . ἔπειτα βαλάνους ἐμβάλλεσθαι τρεῖς μὴ ὁμοτρόπους , τούτων δὲ ἕκαστον φυλάττειν ἀνὰ μίαν τῶν στρατηγῶν · εἰ δὲ πλείονες εἴησαν , πάλῳ καθ’ ἡμέραν τοὺς λαχόντας . ἄριστον δὲ τὰς βαλάνους μὴ ἐξαιρετὰς εἶναι , ὑπὸ δὲ λοπίδος σιδηρᾶς κατέχεσθαι , ἵνα μὴ πλ

In [14]:
#concatenazione dei dataframes di un'opera in un unico dataframe 
#nell'esempio abbiamo la prima opera. 
df1 = pd.concat(dataframes[0],ignore_index=True)
#per farli tutti e metterli in una lista:
dataframes_unificati = []
for i in range(len(dataframes)):
    dataframes_unificati.append(pd.concat(dataframes[i],ignore_index=True))


In [16]:
dataframes_unificati[1]

,@ref,@id,@form,@lemma,@postag,@relation
0,Leuven|0096-00#|1|1,1,τῷ,ὁ,l-s---md-,ATR
1,Leuven|0096-00#|1|2,2,κόρακι,κόραξ,n-s---md-,ADV
2,Leuven|0096-00#|1|3,3,τό,ὁ,l-s---nn-,ATR
3,Leuven|0096-00#|1|4,4,θήραμα,θήραμα,n-s---nn-,SBJ
4,Leuven|0096-00#|1|5,5,τυρός,τυρός,n-s---mn-,PNOM
...,...,...,...,...,...,...
7497,Leuven|0096-00#|1400|12,11,κἄν,κἄν,c--------,AuxC
7498,Leuven|0096-00#|1400|13,12,τά,ὁ,l-p---na-,ATR
7499,Leuven|0096-00#|1400|14,13,μέγιστα,μέγας,a-p---na-,OBJ
7500,Leuven|0096-00#|1400|15,14,φιλανθρωπεύωνται,φιλανθρωπεύομαι,v3ppse---,ADV


In [18]:
#funzione per creare un dizionario in cui come chiave ho la form e come valore il lemma. 
# Prova per pedalion
# Initialize the dictionary
def create_dictonary_of_lemmas(dataframes_unificati):
    df_per_lemma = pd.concat(dataframes_unificati).reset_index(drop=True)[['@id','@form','@lemma','@postag','@relation']]
    lemma_dict = {}

    # Get unique forms from both DataFrames
    forms = list(dict.fromkeys(list(df_per_lemma['@form'])))

    for form in forms:
        # Extract lemmas from df1
        lemmas_df = df_per_lemma.loc[df_per_lemma['@form'] == form, '@lemma'].dropna().unique().tolist()



        # Extract unique @postag values for the form from df1
        postags = df_per_lemma.loc[df_per_lemma['@form'] == form, '@postag'].dropna().unique().tolist()

        # Add to the dictionary
        lemma_dict[form] = {
            'lemmas': lemmas_df,
            'postags': postags
        }
       
    return lemma_dict    


In [19]:
lemma_dic_pedalion = create_dictonary_of_lemmas(dataframes_unificati)

In [21]:
import pickle

In [23]:
with open('lemma_dic_pedialion.pkl', 'wb') as file:
    pickle.dump(lemma_dic_pedalion, file)